In [ ]:
!pip install nltk -q
!pip install sentence-transformers -q
!pip install gradio -q


import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from nltk. corpus import stopwords

stop=stopwords.words('english')

from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')





In [ ]:
## a search engine nlp in unsupervised ## pltk file??


df = pd.read_csv('Data_1.csv')

print(df.shape)
df.head()


(79287, 1)


,title
0,Solving QBF Instances with Nested SAT Solvers.
1,Multiscale Manifold Learning.
2,Online Transfer Learning for Differential Diag...
3,Logic Programming in Assumption-Based Argument...
4,Role-Aware Conformity Modeling and Analysis in...


In [ ]:
sample = df.sample(5000)

sample = sample.reset_index(drop= True)
print(sample.shape)
sample.head()

(5000, 1)


,title
0,"e-Seesaw: A Tangible, Ludic, Parent-child, Awa..."
1,Organizing Hidden-Web Databases by Clustering ...
2,"Management, Sharing and Reuse of Service-Based..."
3,Prices Matter for the Parameterized Complexity...
4,Bidding Strategies for Realistic Multi-Unit Se...


In [ ]:
## cleaning

sample['ctitle'] = sample ['title'].str.replace('[^\w\s]', '')
sample['ctitle'] = sample ['ctitle'].str.replace('\d+', '')  ##remove numbers
sample['ctitle'] = sample ['ctitle'].str.lower()

## stem the words (reducing the words to its root form)

##removing stopwords
sample['ctitle'] = sample['ctitle'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

sample.head()

,title,ctitle
0,"e-Seesaw: A Tangible, Ludic, Parent-child, Awa...",eseesaw tangible ludic parentchild awareness s...
1,Organizing Hidden-Web Databases by Clustering ...,organizing hiddenweb databases clustering visi...
2,"Management, Sharing and Reuse of Service-Based...",management sharing reuse servicebased mobile a...
3,Prices Matter for the Parameterized Complexity...,prices matter parameterized complexity shift b...
4,Bidding Strategies for Realistic Multi-Unit Se...,bidding strategies realistic multiunit sealedb...


In [ ]:
## sentence embedding
##definig as an object
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
##encode converting text to number

features = model.encode(sample['ctitle'])
sample['features'] = list (features)
sample.head()




,title,ctitle,features
0,"e-Seesaw: A Tangible, Ludic, Parent-child, Awa...",eseesaw tangible ludic parentchild awareness s...,"[-1.2207142, -0.3291673, 0.2717755, -0.7450755..."
1,Organizing Hidden-Web Databases by Clustering ...,organizing hiddenweb databases clustering visi...,"[-1.0716456, -0.24232195, 0.2623621, -0.638964..."
2,"Management, Sharing and Reuse of Service-Based...",management sharing reuse servicebased mobile a...,"[-1.0493581, -0.30616596, 0.8406489, -0.899528..."
3,Prices Matter for the Parameterized Complexity...,prices matter parameterized complexity shift b...,"[-1.1421435, -0.54681003, 0.4071678, -0.401826..."
4,Bidding Strategies for Realistic Multi-Unit Se...,bidding strategies realistic multiunit sealedb...,"[-0.44807187, -0.91928625, 0.36515376, 0.06247..."


In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
len(sample.iloc[0]['features'])

768

In [ ]:
sample_query = 'netowrk risk analyis using apriori algorithm'

In [ ]:
query_feature = model.encode([sample_query])
print(query_feature[:100])

In [ ]:
sample['similarity'] = sample['features'] .apply(lambda x: cosine_similarity( [x] , query_feature) [0][0])

In [ ]:
sample.head()

,title,ctitle,features,similarity
0,"e-Seesaw: A Tangible, Ludic, Parent-child, Awa...",eseesaw tangible ludic parentchild awareness s...,"[-1.2207142, -0.3291673, 0.2717755, -0.7450755...",0.507837
1,Organizing Hidden-Web Databases by Clustering ...,organizing hiddenweb databases clustering visi...,"[-1.0716456, -0.24232195, 0.2623621, -0.638964...",0.565471
2,"Management, Sharing and Reuse of Service-Based...",management sharing reuse servicebased mobile a...,"[-1.0493581, -0.30616596, 0.8406489, -0.899528...",0.563202
3,Prices Matter for the Parameterized Complexity...,prices matter parameterized complexity shift b...,"[-1.1421435, -0.54681003, 0.4071678, -0.401826...",0.659486
4,Bidding Strategies for Realistic Multi-Unit Se...,bidding strategies realistic multiunit sealedb...,"[-0.44807187, -0.91928625, 0.36515376, 0.06247...",0.660855


In [ ]:
##arrange a dataframe
sorted_df = sample.sort_values(by = 'similarity', ascending = False )
sorted_df.head()

,title,ctitle,features,similarity
4685,Layered Scene Decomposition via the Occlusion-...,layered scene decomposition via occlusioncrf,"[-0.76866394, -0.5137009, 0.27704504, -0.56695...",0.790088
1261,Online ARIMA Algorithms for Time Series Predic...,online arima algorithms time series prediction,"[-0.7234504, -0.5941054, 0.41460407, -0.813660...",0.788351
1803,Analysing Survey Propagation Guided Decimation...,analysing survey propagation guided decimation...,"[-0.6154081, -0.49252176, -0.09073895, -0.1016...",0.788342
1569,Deriving Non-zeno Behavior Models from Goal Mo...,deriving nonzeno behavior models goal models u...,"[-1.3269526, -0.27786162, 0.38215414, -0.64411...",0.784422
2011,Classification and Analysis of Distributed Eve...,classification analysis distributed event filt...,"[-0.9189836, -0.616953, -0.0008824952, -0.4547...",0.777792


In [ ]:
sample.iloc[1740]['title']

'A computer-vision-assisted system for Videodescription scripting.'

In [ ]:
def search(query):
  query_feature = model.edncode([query])
  sample['similarity'] = sample['features'] .apply(lambda x: cosine_similarity( [x] , query_feature) [0][0])
  sorted_df = sample.sort_values(by = 'similarity', ascending = False )
  return sorted_df.head(10)

In [ ]:
def search(query):
  query_feature = model.encode([query])
  sample['similarity'] = sample['features'] .apply(lambda x: cosine_similarity( [x] , query_feature) [0][0])
  sorted_df = sample.sort_values(by = 'similarity', ascending = False )
  res= sorted_df['title'].head(10)
  return res

In [ ]:
results = search ('instrusion detection')
for item in results:
  print(item)

In [ ]:
# import gradio as gr


# iface = gr.Interface(fn = search, inputs ='text',  outputs ='text' )
# iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://af5ff9bedc803086c1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr


iface = gr.Interface(fn = search, inputs ='text',  outputs ='text' )
iface.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2f56547ced250f8e05.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
search('intrusion detection')

1664    A Background Layer Model for Object Tracking T...
2620    Enhancing the Detection of Code Anomalies with...
2875                        Tracking Adversarial Targets.
665     Unified modeling of attacks, vulnerabilities a...
1466    Quantifying Information Leakage in Process Cal...
1064        Tracking Multiple Objects through Occlusions.
2407                        Adversarial Sequence Tagging.
208                                 Taming the Adversary.
3762    The Security of "One-Block-to-Many" Modes of O...
2052    Automated memory leak detection for production...
Name: title, dtype: object